<h1> This notebook will be mainly used for a final capstone project </h1>

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


In [2]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [7]:
neighborhoods_data = newyork_data['features']

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [15]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Export dataframe to csv, so that I don't have to reload every time

In [17]:
neighborhoods.to_csv('data/neighborhoods.csv', index=False)

In [18]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [19]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [20]:
CLIENT_ID = 'VIRBGFUFQJMOXE353VXGY1T3NDSUZFH04DTLQSMMJG1JJVXU'
CLIENT_SECRET = 'XC3FBDORORXJNANWRPQUI304KZO5VQSD0MQIXAVE4BQUNLZ3'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VIRBGFUFQJMOXE353VXGY1T3NDSUZFH04DTLQSMMJG1JJVXU
CLIENT_SECRET:XC3FBDORORXJNANWRPQUI304KZO5VQSD0MQIXAVE4BQUNLZ3


In [22]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [23]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [24]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [27]:
search_query = 'Music'
radius = 20000
LIMIT=50
print(search_query + ' .... OK!')

Music .... OK!


In [28]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VIRBGFUFQJMOXE353VXGY1T3NDSUZFH04DTLQSMMJG1JJVXU&client_secret=XC3FBDORORXJNANWRPQUI304KZO5VQSD0MQIXAVE4BQUNLZ3&ll=40.7900869,-73.9598295&v=20180605&query=Music&radius=20000&limit=50'

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d6567de00bad700255bcde0'},
 'response': {'venues': [{'id': '4161e400f964a520721d1fe3',
    'name': 'Radio City Music Hall',
    'location': {'address': '1260 Avenue of the Americas',
     'crossStreet': 'at W 50th St',
     'lat': 40.759850056828405,
     'lng': -73.97934397752239,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.759850056828405,
       'lng': -73.97934397752239}],
     'distance': 3746,
     'postalCode': '10020',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['1260 Avenue of the Americas (at W 50th St)',
      'New York, NY 10020',
      'United States']},
    'categories': [{'id': '5032792091d4c4b30a586d5c',
      'name': 'Concert Hall',
      'pluralName': 'Concert Halls',
      'shortName': 'Concert Hall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_',
       'suffix': '.png'},
      'primar

In [30]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4161e400f964a520721d1fe3,Radio City Music Hall,"[{'id': '5032792091d4c4b30a586d5c', 'name': 'C...",v-1566926814,False,1260 Avenue of the Americas,at W 50th St,40.759850,-73.979344,"[{'label': 'display', 'lat': 40.75985005682840...",3746,10020,US,New York,NY,United States,"[1260 Avenue of the Americas (at W 50th St), N...",34845694,NaN
1,4bd482166f649521c7786dec,Diller Quaile School of Music,"[{'id': '4bf58dd8d48988d199941735', 'name': 'C...",v-1566926814,False,24 E 95th St,Madison Ave,40.786644,-73.955068,"[{'label': 'display', 'lat': 40.78664389200099...",554,10128,US,New York,NY,United States,"[24 E 95th St (Madison Ave), New York, NY 1012...",NaN,NaN
2,44127a5ff964a520d2301fe3,Brooklyn Academy of Music (BAM),"[{'id': '4bf58dd8d48988d136941735', 'name': 'O...",v-1566926814,False,30 Lafayette Ave,btwn Ashland Pl & Saint Felix St,40.686415,-73.977658,"[{'label': 'display', 'lat': 40.68641452147685...",11638,11217,US,Brooklyn,NY,United States,[30 Lafayette Ave (btwn Ashland Pl & Saint Fel...,NaN,NaN
3,4aeb7fa7f964a520b9c221e3,Bloomingdale School Of Music,"[{'id': '4bf58dd8d48988d1a8941735', 'name': 'G...",v-1566926814,False,W 108th St,at Broadway,40.803367,-73.968299,"[{'label': 'display', 'lat': 40.80336702998164...",1641,10026,US,New York,NY,United States,"[W 108th St (at Broadway), New York, NY 10026,...",NaN,NaN
4,45682b85f964a520d63d1fe3,Music Box Theatre,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",v-1566926814,False,239 W 45th St,btwn 7th & 8th Ave,40.758798,-73.987240,"[{'label': 'display', 'lat': 40.75879772894823...",4179,10036,US,New York,NY,United States,"[239 W 45th St (btwn 7th & 8th Ave), New York,...",33204676,NaN


In [31]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]



In [35]:
dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Radio City Music Hall,Concert Hall,1260 Avenue of the Americas,at W 50th St,40.759850,-73.979344,"[{'label': 'display', 'lat': 40.75985005682840...",3746,10020,US,New York,NY,United States,"[1260 Avenue of the Americas (at W 50th St), N...",NaN,4161e400f964a520721d1fe3
1,Diller Quaile School of Music,College Arts Building,24 E 95th St,Madison Ave,40.786644,-73.955068,"[{'label': 'display', 'lat': 40.78664389200099...",554,10128,US,New York,NY,United States,"[24 E 95th St (Madison Ave), New York, NY 1012...",NaN,4bd482166f649521c7786dec
2,Brooklyn Academy of Music (BAM),Opera House,30 Lafayette Ave,btwn Ashland Pl & Saint Felix St,40.686415,-73.977658,"[{'label': 'display', 'lat': 40.68641452147685...",11638,11217,US,Brooklyn,NY,United States,[30 Lafayette Ave (btwn Ashland Pl & Saint Fel...,NaN,44127a5ff964a520d2301fe3
3,Bloomingdale School Of Music,General College & University,W 108th St,at Broadway,40.803367,-73.968299,"[{'label': 'display', 'lat': 40.80336702998164...",1641,10026,US,New York,NY,United States,"[W 108th St (at Broadway), New York, NY 10026,...",NaN,4aeb7fa7f964a520b9c221e3
4,Music Box Theatre,Theater,239 W 45th St,btwn 7th & 8th Ave,40.758798,-73.987240,"[{'label': 'display', 'lat': 40.75879772894823...",4179,10036,US,New York,NY,United States,"[239 W 45th St (btwn 7th & 8th Ave), New York,...",NaN,45682b85f964a520d63d1fe3
5,Manhattan School of Music,College Theater,120 Claremont Ave,at Broadway,40.812850,-73.961462,"[{'label': 'display', 'lat': 40.81284958225866...",2537,10027,US,New York,NY,United States,"[120 Claremont Ave (at Broadway), New York, NY...",NaN,4aede9f5f964a5202dd021e3
6,Fiorello H. LaGuardia High School of Music & A...,High School,100 Amsterdam Ave,at 65th St.,40.774048,-73.985012,"[{'label': 'display', 'lat': 40.77404823246509...",2773,10023,US,New York,NY,United States,"[100 Amsterdam Ave (at 65th St.), New York, NY...",NaN,4af1a9c2f964a520f8e121e3
7,Universal Music Group,Office,1755 Broadway,at 57th St.,40.765836,-73.982361,"[{'label': 'display', 'lat': 40.7658357365184,...",3300,10019,US,New York,NY,United States,"[1755 Broadway (at 57th St.), New York, NY 100...",NaN,4a8ebed0f964a5208c1220e3
8,Rockwood Music Hall,Rock Club,196 Allen St,btwn Stanton & E Houston St,40.722196,-73.988841,"[{'label': 'display', 'lat': 40.72219633854976...",7943,10002,US,New York,NY,United States,"[196 Allen St (btwn Stanton & E Houston St), N...",NaN,42b75880f964a52088251fe3
9,Music Hall of Williamsburg,Music Venue,66 N 6th St,at Kent Ave,40.719244,-73.961679,"[{'label': 'display', 'lat': 40.71924372755579...",7887,11249,US,Brooklyn,NY,United States,"[66 N 6th St (at Kent Ave), Brooklyn, NY 11249...",NaN,47048266f964a5204f4b1fe3


In [45]:
venue_types = ['Concert Hall', 'Theater', 'Rock Club', 'Music Venue', 'General Entertainment', 'Opera House']
music_venues = dataframe_filtered.loc[dataframe_filtered['categories'].isin(venue_types)]

In [49]:
music_venues

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Radio City Music Hall,Concert Hall,1260 Avenue of the Americas,at W 50th St,40.759850,-73.979344,"[{'label': 'display', 'lat': 40.75985005682840...",3746,10020,US,New York,NY,United States,"[1260 Avenue of the Americas (at W 50th St), N...",NaN,4161e400f964a520721d1fe3
2,Brooklyn Academy of Music (BAM),Opera House,30 Lafayette Ave,btwn Ashland Pl & Saint Felix St,40.686415,-73.977658,"[{'label': 'display', 'lat': 40.68641452147685...",11638,11217,US,Brooklyn,NY,United States,[30 Lafayette Ave (btwn Ashland Pl & Saint Fel...,NaN,44127a5ff964a520d2301fe3
4,Music Box Theatre,Theater,239 W 45th St,btwn 7th & 8th Ave,40.758798,-73.987240,"[{'label': 'display', 'lat': 40.75879772894823...",4179,10036,US,New York,NY,United States,"[239 W 45th St (btwn 7th & 8th Ave), New York,...",NaN,45682b85f964a520d63d1fe3
8,Rockwood Music Hall,Rock Club,196 Allen St,btwn Stanton & E Houston St,40.722196,-73.988841,"[{'label': 'display', 'lat': 40.72219633854976...",7943,10002,US,New York,NY,United States,"[196 Allen St (btwn Stanton & E Houston St), N...",NaN,42b75880f964a52088251fe3
9,Music Hall of Williamsburg,Music Venue,66 N 6th St,at Kent Ave,40.719244,-73.961679,"[{'label': 'display', 'lat': 40.71924372755579...",7887,11249,US,Brooklyn,NY,United States,"[66 N 6th St (at Kent Ave), Brooklyn, NY 11249...",NaN,47048266f964a5204f4b1fe3
10,"Rockwood Music Hall, Stage 2",Rock Club,196 Allen St,at Houston St,40.722038,-73.988634,"[{'label': 'display', 'lat': 40.722038, 'lng':...",7955,10002,US,New York,NY,United States,"[196 Allen St (at Houston St), New York, NY 10...",NaN,4bc7bcec92b376b06762503a
12,Shrine World Music Venue,Music Venue,2271 Adam Clayton Powell Jr Blvd,134th St,40.814403,-73.944242,"[{'label': 'display', 'lat': 40.81440254987884...",3008,10030,US,New York,NY,United States,"[2271 Adam Clayton Powell Jr Blvd (134th St), ...",NaN,4aed0b8cf964a520ebcc21e3
23,"Rockwood Music Hall, Stage 3",Rock Club,185 Orchard St,Houston Street,40.722100,-73.988225,"[{'label': 'display', 'lat': 40.72209996272042...",7937,10002,US,New York,NY,United States,"[185 Orchard St (Houston Street), New York, NY...",NaN,5081c443e4b0921f5babc957
28,Music on 4,Music Venue,21 W 100th St,NaN,40.794657,-73.963848,"[{'label': 'display', 'lat': 40.79465658051732...",611,10025,US,New York,NY,United States,"[21 W 100th St, New York, NY 10025, United Sta...",NaN,4c898e2e770fb60c2931cbc3
29,Rivington Music Rehearsal Studios,Music Venue,188 Stanton St,btwn Attorney & Ridge,40.719958,-73.983489,"[{'label': 'display', 'lat': 40.71995757133927...",8057,10002,US,New York,NY,United States,"[188 Stanton St (btwn Attorney & Ridge), New Y...",NaN,4ae773b7f964a52070ab21e3


In [52]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Manhattan',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the music venues as blue circle markers
for lat, lng, label in zip(music_venues.lat, music_venues.lng, music_venues.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [53]:
manhattan_data

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


In [54]:
for lat, lng, label in zip(manhattan_data.Latitude, manhattan_data.Longitude, manhattan_data.Neighborhood):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

In [55]:
venues_map